#ДЗ 2 по АОТ
сначала скачаем и импортируем все что нам надо



In [3]:
!pip install natasha
!pip install pymorphy2
!pip install git+https://github.com/nlpub/pymystem3
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install flair
!pip install nltk

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-7sgwvd1p
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-7sgwvd1p
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9922 sha256=2160c3c2ec9029ec67da85953e38b330445e566598a480bd1c6d81eb8099f6f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-l1tnuv7o/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
import pymorphy2
m = pymorphy2.MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize
from pymystem3 import Mystem
m2 = Mystem()
import spacy
from flair.data import Sentence
from flair.models import SequenceTagger
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Добавим тексты:

In [5]:
rutext = '''Я быстро вставила в ухо красивую сережку. Сережка громко засмеялся. На Тянь-Шане было очень холодно. Всю знать позвали на пышный обед. Его френдята на Фейсбуке поставили мне дохера лайков. Калтывый бусик ударил шонку. Кринжовая ситуэйшен произошла со мной на прошлых выходных. Эта девочка была ультраженственной. Френдя знакомых в тиктоке, она испытывала очень странные ощущения. Она тогда была уехалши. Не емши, он вышел на лестничную клетку. Я висю на турнике и пытаюсь подтянуться. Этот новенький деревяннее некуда. Дежурный снова курит айкос. Я в прошлом году поступил в МГУ. Я ездила во Мгу на этих выходных. Она тогда была увезена в тайгу.'''
entext = '''I gollowed the nusk. I will be gollowing this soon. This gollowing will kill me. I don’t have a permit to do this. I watched him killing Eve. I’m waiting for a Christmas Eve to pass. Lying to me won’t make our relationship work. I want to object. This object looks very roundish. I look at his progress every day. I progress slowly in maths. I’ll be home at seventish. The waiting area was huge. I am waiting for you. This made me go numb. This made bed looked lonely. My weekly routine is quite boring. I do this weekly.'''
textn = 'I gollowed the nusk. I will be gollowing this soon. This gollowing will kill me. I do not have a permit to do this. I watched him killing Eve. I am waiting for a Christmas Eve to pass. Lying to me will not make our relationship work. I want to object. This object looks very roundish. I look at his progress every day. I progress slowly in maths. I will be home at seventish. The waiting area was huge. I am waiting for you. This made me go numb. This made bed looked lonely. My weekly routine is quite boring. I do this weekly.'

Почему три текста? Потому что nltk не умеет нормально парсить слова типа don't, и я для него их заменила на do not, а так ничего не меняла.

###Русский:
Я быстро вставила в ухо красивую сережку. Сережка громко засмеялся. На Тянь-Шане было очень холодно. Всю знать позвали на пышный обед. Его френдята на Фейсбуке поставили мне дохера лайков. Калтывый бусик ударил шонку. Кринжовая ситуэйшен произошла со мной на прошлых выходных. Эта девочка была ультраженственной. Френдя знакомых в тиктоке, она испытывала очень странные ощущения. Она тогда была уехалши. Не емши, он вышел на лестничную клетку. Я висю на турнике и пытаюсь подтянуться. Этот новенький деревяннее некуда. Дежурный снова курит айкос. Я в прошлом году поступил в МГУ. Я ездила во Мгу на этих выходных. Она тогда была увезена в тайгу.

Тут разные слова, которые затрудняют парсерам жизнь: слова, которые могут быть разными частями речи, имена собственные, неологизмы, ненастоящие слова, деепричастия, диалектные варианты и другое.

###Английский
I gollowed the nusk. I will be gollowing this soon. This gollowing will kill me. I don’t have a permit to do this. I watched him killing Eve. I’m waiting for a Christmas Eve to pass. Lying to me won’t make our relationship work. I want to object. This object looks very roundish. I look at his progress every day. I progress slowly in maths. I’ll be home at seventish. The waiting area was huge. I am waiting for you. This made me go numb. This made bed looked lonely. My weekly routine is quite boring. I do this weekly.

Тут тоже слова, которые трудно парсить: ненастоящие, существительные-глаголы, имена собственные, разговорные, причастия, местоимения-детерминаторы, прилагательные-наречия.

Мои разборы подгрузим с компа:

In [6]:
def upload(name):
  with open(name, encoding='utf-8') as f:
    pos = f.read()
    s = pos.split('\n')
    poss = [i.split(' ') for i in s]
  return(poss)

In [50]:
rupos = upload('RUPOS.txt')
enpos = upload('ENPOS.txt')

###Наташа


In [52]:
def nat(rutext):
  segmenter = Segmenter()
  emb = NewsEmbedding()
  morph_tagger = NewsMorphTagger(emb)
  syntax_parser = NewsSyntaxParser(emb)
  ner_tagger = NewsNERTagger(emb)
  doc =  Doc(rutext)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  natwordpos = []
  for i in doc.sents:
    for j in i.morph.tokens:
      natwordpos.append([j.text, j.pos])
  return([i for i in natwordpos if i[0] not in ['.', ',']])

###Pymorphy

In [31]:
def pymorphy1(rutext):
  ruwords = simple_word_tokenize(rutext)
  pymwordpos = []
  for i in ruwords:
    pymwordpos.append([i, m.parse(i)[0].tag.POS])
  return([i for i in pymwordpos if i[0] not in ['.', ',']])

###Mystem

In [32]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-13 11:00:49--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm903.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-13 11:00:49--  http://cache-mskm903.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm903.cdn.yandex.net (cache-mskm903.cdn.yandex.net)... 5.45.220.13, 2a02:6b8:0:2002::14
Connecting to cache-mskm903.cdn.yandex.net (cache-mskm903.cdn.yandex.net)|5.45.220.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.2’

mystem-3.0-linux3.1 100%[===================>] 

In [33]:
def myst(rutext):
  ana = m2.analyze(rutext)
  myswordpos = []
  for word in ana:
    if 'analysis' in word:
      gr = word['analysis'][0]['gr']
      pos = gr.split('=')[0].split(',')[0]
      myswordpos.append([word['text'], pos])
  return(myswordpos)

###Spacy


In [49]:
def spacy1(entext):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(entext)
  spawordpos = []
  for i, s in enumerate(doc.sents):
    for t in s:
      spawordpos.append([t.text, t.pos_])
  return([i for i in spawordpos if i[0] != '.'])

###Flair


In [48]:
def flair1(entext):
  sentence = Sentence(entext)
  tagger = SequenceTagger.load('pos')
  tagger.predict(sentence)
  x = sentence.to_tagged_string()
  y = x.split(' ')
  flawordpos = []
  for i in range(0, len(y), 2):
    flawordpos.append([y[i], y[i+1]])
  return([i for i in flawordpos if i[0] != '.'])

###NLTK


In [47]:
def nltk1(textn):
  words = word_tokenize(textn)
  nltkwordpos1 = nltk.pos_tag(words)
  nltkwordpos = []
  for i, j in nltkwordpos1:
    nltkwordpos.append([i, j])
  return([i for i in nltkwordpos if i[0] != '.'])

Напишем функцию, которая будет приводить все тэги к единому виду.

In [84]:
def sync(wp):
  tags = {
    'NOUN': 'NOUN', 
    'PROPN': 'NOUN',
    '<NN>': 'NOUN',
    'ADJF': 'ADJ',
    'ADJ': 'ADJ',
    'ADJS': 'ADJ',
    'COMP': 'ADJ', 
    'VERB': 'VERB',
    'INFN': 'VERB',
    'PRTF': 'VERB',
    'PRTS': 'VERB',
    'GRND': 'VERB',
    'AUX': 'VERB',
    '<VB>': 'VERB',
    '<VBP>': 'VERB',
    '<VBD>': 'VERB',
    '<VBG>': 'VERB',
    '<VBZ>': 'VERB',
    'NUMR': 'NUM',
    'NUM': 'NUM',
    'ADVB': 'ADV',
    'ADV': 'ADV',
    '<RB>': 'ADV',
    'NPRO': 'PRON',
    'PRON': 'PRON',
    'PRED': 'VERB',
    'PREP': 'ADP',
    'ADP' : 'ADP',
    '<PRP>' : 'PRON',
    'CONJ': 'CCONJ',
    'CCONJ': 'CCONJ',
    'PRCL': 'PART',
    'PART': 'PART',
    'INTJ': 'NONE',
    'NONE': 'NONE',
    'PUNCT': 'NONE',
    'X': 'NONE',
    '<.>': 'NONE',
    '.': 'NONE',
    'DET': 'DET',
    '<DT>': 'DET',
    '<JJ>': 'ADJ',
    '<IN>': 'ADP',
    '<TO>': 'PART',
    '<NNP>': 'NOUN',
    '<PRP$>': 'DET',
    '<NNS>': 'NOUN',
    '<,>': 'NONE',
    '<MD>': 'VERB',
    'PRP': 'PRON',
    'DT': 'DET',
    'NN': 'NOUN',
    'VBP': 'VERB',
    'JJ': 'ADJ',
    'VB': 'VERB',
    'VBD': 'VERB',
    'VBG': 'VERB',
    'RB': 'ADV',
    'IN': 'ADP',
    'TO': 'PART',
    'NNS': 'NOUN',
    'NNP': 'NOUN',
    'PRP$': 'DET',
    'MD': 'VERB',
    'VBZ': 'VERB',
    'VBN': 'VERB',
    'S': 'NOUN',
    'V': 'VERB', 
    'PR': 'ADP', 
    'SPRO': 'PRON', 
    'A': 'ADJ',
    'APRO': 'DET', 
    'ADVPRO': 'ADV',
    'CONJ': 'CCONJ',
    None: 'NONE'
  }
  new = [[i[0], tags[i[1]]] for i in wp]
  return(new)


Теперь напишем функцию, которая считает точность.


In [85]:
def accuracy(wp, lang):
  c = 0
  for i in range(len(lang)):
    if wp[i][1]==lang[i][1]:
      c += 1
  return(c/len(lang))

А теперь их все выполним и посчитаем!

In [86]:
nata = sync(nat(rutext))
pym = sync(pymorphy1(rutext))
mys=sync(myst(rutext))
ru = sync(rupos)

spa = sync(spacy1(entext))
fla = sync(flair1(entext))
nlt = sync(nltk1(textn))
en = sync(enpos)

2020-10-13 11:54:59,649 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


In [87]:
print('Наташа '+str(accuracy(nata, ru)))
print('Pymorphy '+str(accuracy(pym, ru)))
print('Mystem '+str(accuracy(mys, ru)))
print('')
print('Spacy '+str(accuracy(spa, en))[:4])
print('Flair '+str(accuracy(fla, en))[:4])
print('NLTK '+str(accuracy(nlt, en))[:4])

Наташа 0.91
Pymorphy 0.81
Mystem 0.92

Spacy 0.92
Flair 0.90
NLTK 0.83


###Выводы
Mystem лучше всех для русского (92%)
Наташа немного остает (91%)
Pymorphy сильно хуже (81%)

Spacy лучше всех для английского (92%)
Flair немного отстает (90%)
NLTK сильно хуже (83%)


###Бонус
Сложные слова: слова, в которых парсеры ошибались.

In [78]:
def hard(lang, f, s, t):
  hard1 = []
  for i in range(len(lang)):
    if not (lang[i][1] == f[i][1] == s[i][1] == t[i][1]):
      hard1.append(lang[i][0])
  return(hard1)

In [83]:
ruhard = hard(ru, nata, pym, mys)
enhard = hard(en, spa, fla, nlt)
print('Сложных русских слов: '+str(len(ruhard)))
print(', '.join(ruhard))
print('Сложных английских слов: '+str(len(enhard)))
print(', '.join(enhard))

Сложных русских слов: 25
холодно, Всю, знать, Его, френдята, дохера, лайков, Калтывый, Кринжовая, ситуэйшен, выходных, Эта, ультраженственной, Френдя, знакомых, уехалши, емши, висю, Этот, новенький, деревяннее, некуда, Дежурный, этих, выходных
Сложных английских слов: 17
this, gollowing, n’t, this, to, n’t, work, object, home, seventish, waiting, This, numb, made, lonely, this, weekly


Показатели точности в процентах для русского и английского близкие, а в русском сложных слов больше, значит, английские парсеры делает ошибки в одних и тех же местах, а русские -  в разных.